In [1]:
import sys
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np
import importlib

sys.path.append('C:\\Users\\kpandit\\PICASSO\\picasso\\')
import mine
importlib.reload(mine)

<module 'mine' from 'C:\\Users\\kpandit\\PICASSO\\picasso\\mine.py'>

In [2]:
import matplotlib.pyplot as plt

px_size = 256
mixing_parameter = 0.5
bit_depth = 12
signal_min = 1000

max_px = 2**bit_depth-1
# Source fluorescence
source_bg = torch.randint(low=90, high=120, size=(px_size,px_size), dtype=torch.int16)  # background
#source_signal = torch.randint(low=signal_min, high=max_px, size=(px_size,px_size), dtype=torch.int16)
px_i = int(px_size/3)
source_signal = torch.zeros_like(source_bg)
source_signal[0:px_i,0:px_i] = 3000
source = source_bg+source_signal
# Sink fluorescence
sink_bg = torch.randint(low=100, high=135, size=(px_size,px_size), dtype=torch.int16)  # background
sink_signal = torch.zeros_like(source_bg)
sink_signal[px_size-px_i:px_size,px_size-px_i:px_size] = 3000
# Spillover fluorecence
spill = mixing_parameter*source_signal       # spillover
spill = spill.type(torch.int16)
sink = sink_bg+sink_signal+spill
sink = sink.clamp(max=4095)


fig, axs = plt.subplots(1,2)
axs[0].imshow(source)
axs[0].set_title('Source')
axs[0].axis('off')
axs[1].imshow(sink)
axs[1].set_title('Sink')
axs[1].axis('off')
            
plt.show()

source = source.flatten()
#source = source.flatten().long()
#source = source.unsqueeze(dim=0)
sink = sink.flatten()
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'
source = source.to(device)
sink = sink.to(device)

<Figure size 640x480 with 2 Axes>

Minimize Mutual Information using Bayesian Optimization

Model: Neural net to transform source fluorescence into spill over fluorescence in the sink channel

Posterior: needs single posterior() method that takes in a Tensor X of design points, and returns a Posterior object describing the (joint) probability distribution of the model output(s) over the design points in X.
Use Joint Distribution of source and sink - network(source) pixel values

Acquisition: Heuristics employed to evaluate the usefulness of one of more design points for achieving the objective of maximizing the underlying black box function.
Use MutualInformation(source, sink - network(source))

In [3]:
from botorch.models.model import Model
from botorch.acquisition.monte_carlo import MCAcquisitionFunction
from botorch.posteriors.deterministic import DeterministicPosterior
import picasso
from picasso import joint_distribution
from botorch.sampling.samplers import MCSampler, SobolQMCNormalSampler
from botorch.utils import t_batch_mode_transform

import picasso
importlib.reload(picasso)
from picasso import joint_distribution

from torch import Tensor
from typing import Optional 



class PICASSObo(Model):
    def __init__(self, transformer = None, min_px=0, max_px = 4095, device=None):
        
        super().__init__()
        
        if torch.cuda.is_available():
            self.device = 'cuda'
        else:
            self.device = 'cpu'
            
        if transformer is None:
            self.tranform_inputs = nn.Sequential(
                nn.Linear(1, 8),
                nn.ReLU(),
                nn.Linear(8, 8),
                nn.ReLU(),
                nn.Linear(8,1),
                nn.Hardtanh(min_val=min_px, max_val=max_px)
            )
        self.tranform_inputs.to(self.device)
        
        self.min_px = torch.tensor(min_px,device=self.device)
        self.max_px = torch.tensor(max_px,device=self.device)
        self.Pxy = None
        
    def posterior(self, source, sink):
        
        self.eval()
        
        print(source.size())
        

        self.Pxy = Pxy
        
        return DeterministicPosterior(self.Pxy[source.long(),:])
    
    def update_Pxy(self, source, sink):
        
        spill = self.transform_inputs(source).long()
        
        minpx = self.min_px
        maxpx = self.max_px
        Pxy, edges = joint_distribution(source, sink-spill, xmin=minpx, xmax=maxpx, ymin=minpx, ymax=maxpx)
        
        if self.Pxy is None:
            self.Pxy = self.posterior(source, sink)
        else:
            Pxy = self.posterior(source, sink)
            self.Pxy = (self.Pxy + Pxy)/2
            
        return DeterministicPosterior(self.Pxy[])
  

class MutualInformation(MCAcquisitionFunction):
    def __init__(
        self,
        model: Model,
        beta: Tensor,
        weights: Tensor,
        sampler: Optional[MCSampler] = None,
    ) -> None:
        # we use the AcquisitionFunction constructor, since that of 
        # MCAcquisitionFunction performs some validity checks that we don't want here
        super(MCAcquisitionFunction, self).__init__(model=model)
        if sampler is None:
            sampler = SobolQMCNormalSampler(num_samples=512, collapse_batch_dims=True)
        self.sampler = sampler
        self.register_buffer("beta", torch.as_tensor(beta))
        self.register_buffer("weights", torch.as_tensor(weights))

    @t_batch_mode_transform()
    def forward(self, source: Tensor, sink:Tensor) -> Tensor:
        """Evaluate mutual information between `X` and `Y`.

        Args:
            XY: A `(b) x q x d`-dim Tensor of `(b)` t-batches with `q` `d`-dim
                design points each.

        Returns:
            Tensor: A `(b)`-dim Tensor of Upper Confidence Bound values at the
                given design points `X`.
        """
        
        posterior = self.model.posterior(source, sink)
        samples = self.sampler(posterior)  # n x b x q x o
        scalarized_samples = samples.matmul(self.weights)  # n x b x q
        mean = posterior.mean  # b x q x o
        scalarized_mean = mean.matmul(self.weights)  # b x q
        ucb_samples = (
            scalarized_mean
            + math.sqrt(self.beta * math.pi / 2)
            * (scalarized_samples - scalarized_mean).abs()
        )
        return ucb_samples.max(dim=-1)[0].mean(dim=0)
    

In [4]:
model = PICASSObo()
sampler = SobolQMCNormalSampler(num_samples=100, collapse_batch_dims=True)

posterior = model.posterior(source, sink)
samples = sampler(posterior)
scalarized_samples = samples.matmul(model.parameters())

torch.Size([65536])


TypeError: matmul(): argument 'other' (position 1) must be Tensor, not generator

In [7]:
samples.size()

torch.Size([100, 65536, 4098])

In [ ]:
transformer = nn.Sequential(
                nn.Linear(1, 1),
                nn.Hardtanh(min_val=0, max_val=4095)
            )
model = PICASSObo(XY)
MCAcquisitionFunction(model, objective=mutual_)